In [1]:
from nested_cv import NestedCV

import pandas as pd
import numpy as np
from sklearn.datasets import load_boston, load_iris, load_breast_cancer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# When using Random Search, we get a user warning with this little number of hyperparameters
# Suppress it
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Regression Example

In [2]:
boston = load_boston()
X = boston.data
y = boston.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10]
}

NCV = NestedCV(model=RandomForestRegressor(), params_grid=param_grid,
               outer_kfolds=5, inner_kfolds=5, n_jobs = -1,
               cv_options={'sqrt_of_score':True, 
                           'recursive_feature_elimination':True, 
                           'rfe_n_features':2})
NCV.fit(X=X,y=y)

NCV.outer_scores

[3.8731555751351503 3.3742141288979153 4.830080881316246 3.520547972218438
 3.594803883826222 3.212454351887815 4.474999492920611 4.54115868718791
 3.724566172483345 3.4801962947512024]
3.212454351887815
{'max_depth': None, 'n_estimators': 10}
[5.306462055524935 4.258174853903929 3.47923083698778 3.2474077030954174
 4.344633995325251 3.447419503711131 3.8754888659265383 3.511474840512965
 4.4973367677807 3.8302536743506357]
3.2474077030954174
{'max_depth': None, 'n_estimators': 10}
[4.2573598538583175 3.6572247117989067 3.531448194252269 3.332490263755424
 3.8168474313786516 3.306822166295939 3.316329408176702 2.9956831080657746
 3.7119512164191986 3.427606154536616]
2.9956831080657746
{'max_depth': None, 'n_estimators': 10}
[4.087946590826774 3.5403232746302824 4.600968208305476 4.008745378066913
 4.109036561944618 3.1944077292576076 4.465022282937201 3.753237450258735
 3.8620388542178277 3.1212641867449786]
3.1212641867449786
{'max_depth': None, 'n_estimators': 10}
[3.927108998211049

[3.7341606663817966,
 3.3876013009587504,
 5.8785022002897875,
 3.012809615986084,
 3.065403079466705]

# Classification Example

## Breast Cancer (2 Classes)

In [3]:
from sklearn.metrics import roc_auc_score

# Binary classification
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10, 20]
}

NCV = NestedCV(model=RandomForestClassifier(), params_grid=param_grid,
               outer_kfolds=5, inner_kfolds=5,
               cv_options={'metric':roc_auc_score, 
                           'metric_score_indicator_lower':False,
                           'randomized_search_iter':30, 
                           'predict_proba':True})
NCV.fit(X=X,y=y)

NCV.outer_scores

[0.9768244575936884 0.9783037475345168 0.9672090729783037
 0.9842209072978304 0.9891640866873065 0.9943240454076367
 0.9852941176470589 0.9940660474716203 0.992769744160178
 0.9949944382647385 0.9949944382647387 0.9986095661846496
 0.9731182795698925 0.9924731182795699 0.9755376344086022
 0.9760752688172043 0.9793942403177756 0.9771598808341607
 0.9779046673286991 0.9784011916583913]
0.9986095661846496
{'max_depth': None, 'n_estimators': 20}
[0.9984326018808777 0.9989550679205851 0.997387669801463 0.996342737722048
 0.9804075235109717 0.9743991640543365 0.990073145245559
 0.9905956112852664 0.9948979591836735 0.9937641723356009
 0.9880952380952381 0.9982993197278911 0.9884884884884885
 0.9944944944944945 0.9917417417417418 0.9804804804804805
 0.9955312810327706 0.9950347567030784 0.9828699106256207
 0.9952830188679245]
0.9989550679205851
{'max_depth': 3, 'n_estimators': 20}
[0.957172342621259 0.9625902992776058 0.9643962848297214
 0.9760061919504643 0.9901639344262295 0.998907103825136

[0.9929836284664217,
 0.96053062561415,
 0.9921875,
 0.9955781198820832,
 0.9981684981684981]

## Iris (3 Classes)

In [4]:
from sklearn.metrics import f1_score

# Multiclass classification
iris = load_iris()
X = iris.data
y = iris.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10, 20]
}

NCV = NestedCV(model=RandomForestClassifier(), params_grid=param_grid, 
               outer_kfolds=5, inner_kfolds=5,
               cv_options={'metric':f1_score, 
                           'metric_score_indicator_lower':False,
                           'randomized_search_iter':30, 
                           'predict_proba':False,
                           'multiclass_average': 'micro'})
NCV.fit(X=X,y=y)

NCV.outer_scores

[0.9583333333333334 0.9583333333333334 0.9583333333333334
 0.9583333333333334 0.9583333333333334 0.9166666666666666 0.875
 0.9166666666666666 0.9583333333333334 0.9166666666666666
 0.9166666666666666 0.9166666666666666 0.9166666666666666
 0.9166666666666666 0.9583333333333334 0.9583333333333334
 0.9583333333333334 0.9583333333333334 0.9583333333333334
 0.9583333333333334]
0.9583333333333334
{'max_depth': 3, 'n_estimators': 10}
[0.9166666666666666 1.0 0.9166666666666666 0.9166666666666666
 0.9166666666666666 0.9166666666666666 0.9166666666666666
 0.9166666666666666 0.9583333333333334 0.9583333333333334
 0.9166666666666666 0.9166666666666666 0.9583333333333334
 0.9583333333333334 0.9583333333333334 0.875 0.9583333333333334
 0.9583333333333334 0.9583333333333334 0.9583333333333334]
1.0
{'max_depth': 3, 'n_estimators': 20}
[0.875 0.9166666666666666 0.9166666666666666 0.9166666666666666
 0.9583333333333334 0.9583333333333334 0.9583333333333334
 0.9583333333333334 0.9166666666666666 0.958333

[0.9666666666666667,
 0.9666666666666667,
 1.0,
 0.9333333333333333,
 0.9333333333333333]